# Test-Daten Generierung
In diesem Notebook werden Testdaten Paare für die verschiedenen Anforderungen definiert
Die Beispieldaten würden jeweils mehrfmals angepasst um möglichst unterschiedliche Testdaten zu generieren

## Test daten für die Erkennung einer sinvollen Location abfolge

In [60]:
import random
import csv
import math

def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def sort_locations_by_distance(locations):
    remaining = locations[:]
    sorted_locations = [remaining.pop(0)]
    while remaining:
        last = sorted_locations[-1]
        next_loc = min(remaining, key=lambda loc: calculate_distance(last[1], loc[1]))
        sorted_locations.append(next_loc)
        remaining.remove(next_loc)
    return sorted_locations

def generate_names(num_names):
    first_names = ["Alex", "Sam", "Jamie", "Jordan", "Taylor", "Morgan", "Casey", "River", "Skyler", "Riley"]
    last_names = ["Smith", "Johnson", "Williams", "Jones", "Brown", "Davis", "Miller", "Wilson", "Moore", "Taylor"]
    return [random.choice(first_names) + " " + random.choice(last_names) for _ in range(num_names)]

def generate_equipment(num_items):
    equipment_types = ["Camera", "Microphone", "Gimbal", "Drone", "Boom Mic"]
    equipment_labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]
    return [random.choice(equipment_types) + " " + random.choice(equipment_labels) for _ in range(num_items)]

def generate_locations():
    base_locations = [
       ("Schlieren", (47.3962, 8.4515)),
       ("Altstetten", (47.3914, 8.4886)),
       ("Zürich Hardbrücke", (47.3850, 8.5172)),
       ("Zürich Bahnhofstrasse", (47.3733, 8.5377)),
       ("Wollishofen", (47.3407, 8.5310)),
       ("Kilchberg", (47.3255, 8.5407)),
       ("Horgen", (47.2558, 8.5986))
    ]
    return sort_locations_by_distance(base_locations)

def create_schedule():
    num_scenes = 5
    actors = [generate_names(random.randint(2, 4)) for _ in range(num_scenes)]
    equipment = [generate_equipment(random.randint(2, 5)) for _ in range(num_scenes)]
    
    locations = generate_locations()[:num_scenes]
    
    durations = [random.randint(1, 3) for _ in range(num_scenes)]

    scenes = [{'scene_number': i+1, 'actors': a, 'equipment': e, 'location': loc[0], 'coordinates': loc[1], 'duration': d} for i, (a, e, loc, d) in enumerate(zip(actors, equipment, locations, durations))]
    return scenes

def generate_schedule_pairs(num_pairs):
    pairs = []
    for _ in range(num_pairs):
        scenes = create_schedule()
        base_schedule = scenes[:]  # Logical Schedule (unchanged order)
        hopping_schedule = scenes[:]  # Create a copy for the Hopping Schedule
        random.shuffle(hopping_schedule)  # Shuffle the Hopping Schedule
        pairs.append((base_schedule, hopping_schedule))
    return pairs

def export_to_csv(schedule_pairs, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Pair ID', 'Schedule Type', 'Order Number', 'Scene Number', 'Actors', 'Equipment', 'Location', 'Coordinates', 'Duration'])
        for pair_id, (logical, hopping) in enumerate(schedule_pairs, start=1):
            for index, schedule in enumerate([logical, hopping]):
                schedule_type = 'Schedule Plan 1' if index == 0 else 'Schedule Plan 2'
                for order_number, scene in enumerate(schedule, start=1):
                    writer.writerow([
                        pair_id, schedule_type, order_number, f"Scene {scene['scene_number']}", ', '.join(scene['actors']),
                        ', '.join(scene['equipment']), scene['location'], f"{scene['coordinates']}", f"{scene['duration']}h"
                    ])

# Example usage
filename = 'testdata_location.csv'
schedule_pairs = generate_schedule_pairs(10)
export_to_csv(schedule_pairs, filename)
print(f"Schedules have been generated and exported to {filename}.")


Schedules have been generated and exported to schedule_pairs_location.csv.


## Test daten für die Erkennung einer sinvollen Actors/Equipment abfolge

In [59]:
import random
import csv

def generate_names():
    return [
        "Skyler Taylor", "Jamie Taylor", "Morgan Lee", "Taylor Smith", "Casey Johnson",
        "Riley Brown", "River Wilson", "Jordan Davis"
    ]

def generate_equipment():
    return [
        "Microphone C", "Drone J", "Camera D", "Boom Mic D", "Drone H",
        "Gimbal H", "Camera J", "Microphone D"
    ]

def generate_locations():
    return [
        "Zürich Zentrum", "Bahnhofstrasse", "Bellevue"
    ]

def create_base_schedule():
    num_scenes = 5
    actors_list = generate_names()
    equipment_list = generate_equipment()
    locations = generate_locations()
    durations = [random.randint(1, 3) for _ in range(num_scenes)]

    scenes = []
    for i in range(num_scenes):
        scene_actors = random.sample(actors_list, 3)
        scene_equipment = random.sample(equipment_list, 3)
        scenes.append({
            'scene_number': i+1,
            'actors': scene_actors,
            'equipment': scene_equipment,
            'location': locations[i % len(locations)],
            'duration': durations[i]
        })
    return scenes

def shuffle_scenes(scenes):
    shuffled = scenes[:]
    while True:
        random.shuffle(shuffled)
        if all(shuffled[i]['actors'] != shuffled[i+1]['actors'] and shuffled[i]['equipment'] != shuffled[i+1]['equipment'] for i in range(len(shuffled)-1)):
            break
    return shuffled

def generate_schedule_pairs(num_pairs):
    pairs = []
    for _ in range(num_pairs):
        base_schedule = create_base_schedule()
        sequential_schedule = base_schedule[:]
        random_schedule = shuffle_scenes(base_schedule[:])

        pairs.append((sequential_schedule, random_schedule))
    return pairs

def export_to_csv(schedule_pairs, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Pair ID', 'Schedule Type', 'Order Number', 'Scene Number', 'Actors', 'Equipment', 'Location', 'Coordinates', 'Duration'])
        for pair_id, (sequential, random_schedule) in enumerate(schedule_pairs, start=1):
            for schedule, name in zip((sequential, random_schedule), ('Schedule Plan 1', 'Schedule Plan 2')):
                for order_number, scene in enumerate(schedule, start=1):
                    writer.writerow([
                        pair_id, name, order_number, f"Scene {scene['scene_number']}",
                        ', '.join(scene['actors']),
                        ', '.join(scene['equipment']), scene['location'], f"{scene['duration']}h"
                    ])

# Example usage
filename = 'testdata_actors_equipment.csv'
schedule_pairs = generate_schedule_pairs(10)
export_to_csv(schedule_pairs, filename)
print(f"Schedules have been generated and exported to {filename}.")



Schedules have been generated and exported to schedule_pairs_actors_equipment.csv.
